In [ ]:
import os
import time
import numpy as np
# from functools import partial

# import optuna

import confmain
from util_module import util_func
from util_module import model_func
from util_module.ecg_signal import ECGSignal

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
data = util_func.open_pickle('../data/ludb_processed/ludb_i_ii_iii.pickle')

features = data['features']
y = np.array(data['labels'])

#### Splitting to train, validation, and test set

In [ ]:
features_train, features_val, features_test, y_train, y_val, y_test = model_func.train_val_test_split(features, y)

In [ ]:
X_train, X_val, X_test, zpad_length_train, zpad_length_val, zpad_length_test = model_func.get_x(features_train, features_val, features_test)

In [ ]:
X_train.shape

In [ ]:
len(X_train), len(X_val), len(X_test)

#### Hyperparameter tuning

Using CMA-ES (Covariance Matrix Adaptation - Evolution Strategy), implementation provided by optuna.<br>
<br>
CMA-ES objective will be to minimize validation loss.

In [ ]:
def plot_segments(X, y, zpad, idx, save_path):
    signal = X[idx].flatten()
    segment_map = y[idx].argmax(axis=1)

    beat_span = len(signal) - zpad[idx]

    signal = signal[:beat_span]
    segment_map = segment_map[:beat_span]

    ECGSignal.plot_signal_segments(signal, segment_map, save_path)

In [ ]:
BATCH_SIZES = [8, 16, 32, 64]

# def objective(trial, X_train, X_val, X_test, y_train, y_val, y_test):
# # def objective(trial):
#     start_time = time.time()

#     lr = np.round(trial.suggest_float('lr', 1e-5, 1e-3, log=True), decimals=5)
#     n_layer = trial.suggest_int('n_layer', 1, 5)
#     batch_size_index = trial.suggest_int('batch_size_index', 0, len(BATCH_SIZES)-1)
#     bs = BATCH_SIZES[batch_size_index]

#     model_name = f'ConvBiLSTM-LR_{lr}-Nlayer_{n_layer}-BS_{bs}'
#     result_path = f'../result/{model_name}'

#     model = model_func.generate_model((X_train.shape[1], 1), 8, lr=lr, n_layer=n_layer)
#     history = model.fit(X_train, y_train, epochs=200, batch_size=bs, validation_data=(X_val, y_val))

#     model_func.plot_acc_loss(history, result_path)

#     # PREDICT TRAIN
#     save_to_train = f'{result_path}/train'
#     y_pred_train = model.predict(X_train)
#     model_func.calc_metrics(y_train, y_pred_train, save_to_train)
#     model_func.roc_pr(y_train, y_pred_train, save_to_train)
    
#     util_func.make_dir(f'{save_to_train}/delineation')
#     plot_segments(X_train, y_train, zpad=zpad_length_train, idx=0, save_path=f'{save_to_train}/delineation/Expert_annotated.jpg') # Expert annotated
#     plot_segments(X_train, y_pred_train, zpad=zpad_length_train, idx=0, save_path=f'{save_to_train}/delineation/Prediction.jpg') # Prediction
#     # ====================================================

#     # PREDICT VALIDATION
#     save_to_val = f'{result_path}/val'
#     y_pred_val = model.predict(X_val)
#     model_func.calc_metrics(y_val, y_pred_val, save_to_val)
#     model_func.roc_pr(y_val, y_pred_val, save_to_val)

#     util_func.make_dir(f'{save_to_val}/delineation')
#     plot_segments(X_val, y_val, zpad=zpad_length_val, idx=0, save_path=f'{save_to_val}/delineation/Expert_annotated.jpg') # Expert annotated
#     plot_segments(X_val, y_pred_val, zpad=zpad_length_val, idx=0, save_path=f'{save_to_val}/delineation/Prediction.jpg') # Prediction
#     # ====================================================

#     # PREDICT test
#     save_to_test = f'{result_path}/test'
#     y_pred_test = model.predict(X_test)
#     model_func.calc_metrics(y_test, y_pred_test, save_to_test)
#     model_func.roc_pr(y_test, y_pred_test, save_to_test)

#     util_func.make_dir(f'{save_to_test}/delineation')
#     plot_segments(X_test, y_test, zpad=zpad_length_test, idx=0, save_path=f'{save_to_test}/delineation/Expert_annotated.jpg') # Expert annotated
#     plot_segments(X_test, y_pred_test, zpad=zpad_length_test, idx=0, save_path=f'{save_to_test}/delineation/Prediction.jpg') # Prediction
#     # ====================================================

#     val_loss = history.history['val_loss'][-1]
#     if np.isnan(val_loss):
#         val_loss_score = 1 # High enough to not be picked
#     else:
#         val_loss_score = val_loss

#     end_time = time.time()
#     time_elapsed = end_time - start_time
#     with open(f'{result_path}/Model_info.txt', 'w') as info_file:
#         info_file.write(f'{model_name} | Last epoch val loss: {val_loss_score}\n')
#         info_file.write(f'Learning Rate: {lr} | n_layer: {n_layer} | Batch Size: {bs}\n')
#         info_file.write(f'Time elapsed: {time_elapsed:.2f} seconds\n')

#     return val_loss_score

In [ ]:
# objective_wrapper = partial(
#     objective,
#     X_train=X_train,X_val=X_val, X_test=X_test,
#     y_train=y_train, y_val=y_val, y_test=y_test
# )
# sampler = optuna.samplers.CmaEsSampler()
# study = optuna.create_study(study_name='cma-es', sampler=sampler, direction='minimize')
# study.optimize(objective_wrapper, n_trials=20)

In [ ]:
def generate_results(X_train, X_val, X_test, y_train, y_val, y_test, lead):
    start_time = time.time()

    # Good fit model architecture
    lr = 0
    n_layer = 0
    bs = 0
    epoch = 0

    model_name = f'ConvBiLSTM--{lead}--LR_{lr}-Nlayer_{n_layer}-BS_{bs}-Epoch_{epoch}'
    result_path = f'../result/{model_name}'
    save_to_train = f'{result_path}/train'
    save_to_val = f'{result_path}/val'
    save_to_test = f'{result_path}/test'

    util_func.make_dir(f'{save_to_train}/delineation')
    util_func.make_dir(f'{save_to_val}/delineation')
    util_func.make_dir(f'{save_to_test}/delineation')


    model = model_func.generate_model((X_train.shape[1], 1), 8, lr=lr, n_layer=n_layer)
    history = model.fit(X_train, y_train, epochs=epoch, batch_size=bs, validation_data=(X_val, y_val))

    model.save(f'{model_name}.h5')
    model_func.plot_acc_loss(history, result_path)

    # PREDICT TRAIN
    y_pred_train = model.predict(X_train)
    model_func.calc_metrics(y_train, y_pred_train, save_to_train)
    model_func.roc_pr(y_train, y_pred_train, save_to_train)
    
    for i in range(0, 41, 10):
        plot_segments(X_train, y_train, zpad=zpad_length_train, idx=i, save_path=f'{save_to_train}/delineation/Expert_annotated_{i}.jpg') # Expert annotated
        plot_segments(X_train, y_pred_train, zpad=zpad_length_train, idx=i, save_path=f'{save_to_train}/delineation/Prediction_{i}.jpg') # Prediction
    # ====================================================

    # PREDICT VALIDATION
    y_pred_val = model.predict(X_val)
    model_func.calc_metrics(y_val, y_pred_val, save_to_val)
    model_func.roc_pr(y_val, y_pred_val, save_to_val)

    for i in range(0, 41, 10):
        plot_segments(X_val, y_val, zpad=zpad_length_val, idx=i, save_path=f'{save_to_val}/delineation/Expert_annotated_{i}.jpg') # Expert annotated
        plot_segments(X_val, y_pred_val, zpad=zpad_length_val, idx=i, save_path=f'{save_to_val}/delineation/Prediction_{i}.jpg') # Prediction
    # ====================================================

    # PREDICT test
    y_pred_test = model.predict(X_test)
    model_func.calc_metrics(y_test, y_pred_test, save_to_test)
    model_func.roc_pr(y_test, y_pred_test, save_to_test)

    for i in range(0, 41, 10):
        plot_segments(X_test, y_test, zpad=zpad_length_test, idx=i, save_path=f'{save_to_test}/delineation/Expert_annotated_{i}.jpg') # Expert annotated
        plot_segments(X_test, y_pred_test, zpad=zpad_length_test, idx=i, save_path=f'{save_to_test}/delineation/Prediction_{i}.jpg') # Prediction
    # ====================================================

    end_time = time.time()
    time_elapsed = end_time - start_time
    with open(f'{result_path}/Model_info.txt', 'w') as info_file:
        info_file.write(f'{model_name}\n')
        info_file.write(f'Learning Rate: {lr} | n_layer: {n_layer} | Batch Size: {bs}\n | Epoch: {epoch}')
        info_file.write(f'Time elapsed: {time_elapsed:.2f} seconds\n')

In [ ]:
# generate_results(X_train, X_val, X_test, y_train, y_val, y_test, lead='i_ii_iii')

In [ ]:
# data = util_func.open_pickle('../data/ludb_processed/ludb_i.pickle')

# features = data['features']
# y = np.array(data['labels'])

# features_train, features_val, features_test, y_train, y_val, y_test = model_func.train_val_test_split(features, y)

# X_train, X_val, X_test, zpad_length_train, zpad_length_val, zpad_length_test = model_func.get_x(features_train, features_val, features_test)

# generate_results(X_train, X_val, X_test, y_train, y_val, y_test, lead='i')

In [ ]:
# data = util_func.open_pickle('../data/ludb_processed/ludb_ii.pickle')

# features = data['features']
# y = np.array(data['labels'])

# features_train, features_val, features_test, y_train, y_val, y_test = model_func.train_val_test_split(features, y)

# X_train, X_val, X_test, zpad_length_train, zpad_length_val, zpad_length_test = model_func.get_x(features_train, features_val, features_test)

# generate_results(X_train, X_val, X_test, y_train, y_val, y_test, lead='ii')

In [ ]:
# data = util_func.open_pickle('../data/ludb_processed/ludb_iii.pickle')

# features = data['features']
# y = np.array(data['labels'])

# features_train, features_val, features_test, y_train, y_val, y_test = model_func.train_val_test_split(features, y)

# X_train, X_val, X_test, zpad_length_train, zpad_length_val, zpad_length_test = model_func.get_x(features_train, features_val, features_test)

# generate_results(X_train, X_val, X_test, y_train, y_val, y_test, lead='iii')